# Libraries
1. [filterpy](https://filterpy.readthedocs.io/en/latest/kalman/UnscentedKalmanFilter.html)

In [1]:
import numpy as np
import pandas as pd
import random
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px
import seaborn as sns

import geopy
import pymap3d as pm
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

# Hyper Parameters

In [2]:
SEED = 1990
random.seed(SEED)
np.random.seed(SEED)

In [3]:
q = np.array([1,1,1,1,1])
r = np.array([1,1,1,1])

# Useful functions

In [4]:
def calc_haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    
    c = 2 * np.arcsin(a ** 0.5)
    dist = 6_367_000 * c
    return dist

In [5]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    
    return output_df, meter_score , score

In [6]:
ell_wgs84 = pm.Ellipsoid()
def calc_geo2enu(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    llh = np.array(df[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']])
    denu = pm.geodetic2enu(llh[:,0], llh[:,1], llh[:,2], llh[0,0], llh[0,1], llh[0,2], ell=ell_wgs84)
    output['x'] = denu[0]
    output['y'] = denu[1]
    output['z'] = denu[2]
    
    return output

def calc_enu2geo(df:pd.DataFrame)->pd.DataFrame:
    output = df.copy()
    enu = np.array(df[['x', 'y', 'z']])
    llh = np.array(df[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']])
    geo = pm.enu2geodetic(enu[:,0], enu[:,1], enu[:,2], llh[0,0], llh[0,1], llh[0,2], ell=ell_wgs84, deg = True)
    output['latDeg'] = geo[0]
    output['lngDeg'] = geo[1]
    output['heightAboveWgs84EllipsoidM'] = geo[2]
    
    return output

# Data

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")
df_train = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [8]:
phones = df_train['phone'].unique()
phone = phones[random.randint(0, len(phones))]
df_sample = df_train[df_train['phone'] == phone].copy().reset_index().drop(columns = ['index'])
print(df_sample.shape)
df_sample.head()

(1746, 148)


,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,GPS_L1,GPS_L5,GAL_E1,GAL_E5A,GLO_G1,BDS_B1I,BDS_B1C,BDS_B2A,QZS_J1,QZS_J5
0,2020-05-14-US-MTV-1,Pixel4XLModded,1273529466449,37.423574,-122.094137,-33.20,2020-05-14-US-MTV-1_Pixel4XLModded,554.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2020-05-14-US-MTV-1,Pixel4XLModded,1273529467449,37.423573,-122.094153,-33.92,2020-05-14-US-MTV-1_Pixel4XLModded,555.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2020-05-14-US-MTV-1,Pixel4XLModded,1273529468449,37.423578,-122.094148,-33.33,2020-05-14-US-MTV-1_Pixel4XLModded,556.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2020-05-14-US-MTV-1,Pixel4XLModded,1273529469449,37.423573,-122.094150,-32.85,2020-05-14-US-MTV-1_Pixel4XLModded,557.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2020-05-14-US-MTV-1,Pixel4XLModded,1273529470449,37.423573,-122.094147,-31.26,2020-05-14-US-MTV-1_Pixel4XLModded,558.45,1.2,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
for col in df_train.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
elapsedRealtimeNanos
yawDeg
rollDeg
pitchDeg
utcTimeMillis_Status
SignalCount
SignalIndex
ConstellationType
Svid
CarrierFrequencyHz
Cn0DbHz
AzimuthDegrees
ElevationDegrees
UsedInFix
HasAlmanacData
HasEphemerisData
BasebandCn0DbHz
utcTimeMillis_UncalMag
elapsedRealtimeNanos_UncalMag
UncalMagXMicroT
UncalMagYMicroT
UncalMagZMicroT
BiasXMicroT
BiasYMicroT
BiasZMicroT
utcTimeMillis_UncalAccel
elapsedRealtimeNanos_UncalAccel
UncalAccelXMps2
UncalAccelYMps2
UncalAccelZMps2
BiasXMps2
BiasYMps2
BiasZMps2
utcTimeMillis_UncalGyro
elapsedRealtimeNanos_UncalGyro
UncalGyroXRadPerSec
UncalGyroYRadPerSec
U

## Model Define
$$
\begin{matrix}
x_t =& x_{t-1} + \frac{v_{t-1}}{w_{t-1}}\left({\sin}\left({\omega}_{t-1}dt + {\theta}_{t-1}\right) - {\sin}\left({\theta}\right)\right)\\
y_t =& y_{t-1} + \frac{v_{t-1}}{w_{t-1}}\left({\cos}\left({\theta}_{t-1}\right) - {\cos}\left({\omega}_{t-1}dt + {\theta}_{t-1}\right)\right)\\
v_t =& v_{t-1}\\
{\theta}_t =& {\theta}_{t-1} + {\omega}_{t-1}dt\\
{\omega}_t =& {\omega}_{t-1}
\end{matrix}
$$

In [10]:
def fx(x, dt):
    xout = np.zeros_like(x)
    if abs(x[4]) > 1e-3:
        xout[0] = x[0] + x[2]/x[4] * (np.sin(x[4] * dt + x[3]) - np.sin(x[3]))
        xout[1] = x[1] + x[2]/x[4] * (np.cos(x[3]) - np.cos(x[4] * dt + x[3]))
        xout[2] = x[2]
        xout[3] = x[3] + x[4] * dt
        xout[4] = x[4]
    else:
        xout[0] = x[0] + x[2] * dt * (np.cos(x[3]))
        xout[1] = x[1] + x[2] * dt * (np.sin(x[3]))
        xout[2] = x[2]
        xout[3] = x[3] + x[4] * dt
        xout[4] = x[4]
        
    return xout

In [11]:
def batch_filter(df, q_, r_):
    df1 = calc_geo2enu(df)
    df1['yawRad'] = np.deg2rad(df1['yawDeg'])
    
    
    features = ['x', 'y']
    index = [0, 1]
    rindex = [0, 1]
    if df1['yawDeg'].isna().mean() == 1 or df1['yawDeg'].mean() == 0:
        pass
    else:
        features += ['yawRad']
        index += [3]
        rindex+= [2]
    if df1['UncalGyroZRadPerSec'].isna().mean() == 1 or df1['UncalGyroZRadPerSec'].mean() == 0:
        pass
    else:
        features += ['UncalGyroZRadPerSec']
        index += [4]
        rindex+= [3]
    
    q = q_
    r = r_[rindex]
    meas = df1[features]
    meas = meas.fillna(0)
    h = lambda x:x[index]
    
    points = MerweScaledSigmaPoints(5, alpha = .1, beta = 2., kappa = -1)
    kf = UnscentedKalmanFilter(dim_x = 5, dim_z = len(features), dt = 1, fx = fx, hx = h, points = points)
    kf.Q = np.diag(q)
    kf.R = np.diag(r)
    
    mu, cov = kf.batch_filter(meas.values)
    (xs, Ps, Ks) = kf.rts_smoother(mu, cov)
    df2 = df1.copy()
    df2['x'] = xs[:,0]
    df2['y'] = xs[:,1]
    df2['yawDeg'] = np.rad2deg(xs[:,2])
    df2['UncalGyroZRadPerSec'] = xs[:,3]
    df3 = calc_enu2geo(df2)
    return df3

In [12]:
def evaluate(df, q_, r_, get_score = True):
    output = df.copy()
    mean_before, score_before, mean_after, score_after =0, 0, 0, 0
    output.drop(columns = ['latDeg', 'lngDeg'], inplace = True)
    df_list = []
    
    for phone in tqdm(df['phone'].unique()):
        df1 = df[df['phone'] == phone].copy()
        df2 = batch_filter(df1, q_, r_)
        df_list.append(df2[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']])
    df3 = pd.concat(df_list)
    output = output.merge(df3, on = ['phone', 'millisSinceGpsEpoch'])

    if get_score:
        _, mean_before, score_before = check_score(df)
        _, mean_after, score_after = check_score(output)
    
    return output, mean_before, score_before, mean_after, score_after

  0%|          | 0/73 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
bins = 100
fig, axes = plt.subplots(8,1,figsize = (16,32))

axes[0].plot(df_sample['latDeg'])
axes[0].plot(df_sample_filter['latDeg'])

axes[1].plot(df_sample['t_latDeg'] - df_sample['latDeg'])
axes[1].plot(df_sample['t_latDeg'] - df_sample_filter['latDeg'])

axes[2].hist(df_sample['t_latDeg'] - df_sample['latDeg'], alpha = 1/3, bins = bins)
axes[2].hist(df_sample['t_latDeg'] - df_sample_filter['latDeg'], alpha = 1/3, bins = bins)

axes[3].plot(df_sample['lngDeg'])
axes[3].plot(df_sample_filter['lngDeg'])

axes[4].plot(df_sample['t_lngDeg'] - df_sample['lngDeg'])
axes[4].plot(df_sample['t_lngDeg'] - df_sample_filter['lngDeg'])

axes[5].hist(df_sample['t_lngDeg'] - df_sample['lngDeg'], alpha = 1/3, bins = bins)
axes[5].hist(df_sample['t_lngDeg'] - df_sample_filter['lngDeg'], alpha = 1/3, bins = bins)

axes[6].plot(df_sample['yawDeg'])
axes[6].plot(df_sample_filter['yawDeg'])

axes[7].plot(df_sample['UncalGyroZRadPerSec'])
axes[7].plot(df_sample_filter['UncalGyroZRadPerSec'])

In [ ]:
for phone in phones:
    df1 = df_train[df_train['phone'] == phone].copy()
    df2 = batch_filter(df1, q, r)
    _, res1_mean , res1_score = check_score(df1)
    _, res2_mean , res2_score = check_score(df2)
    print(phone + " Result")
    print(f"Before: {res1_mean:.4f}, {res1_score:.4f}")
    print(f"After : {res2_mean:.4f}, {res2_score:.4f}")
    print()

In [ ]:
bins = 100
fig, axes = plt.subplots(8,1,figsize = (16,32))

axes[0].plot(df_sample['latDeg'])
axes[0].plot(df_sample_filter['latDeg'])

axes[1].plot(df_sample['t_latDeg'] - df_sample['latDeg'])
axes[1].plot(df_sample['t_latDeg'] - df_sample_filter['latDeg'])

axes[2].hist(df_sample['t_latDeg'] - df_sample['latDeg'], alpha = 1/3, bins = bins)
axes[2].hist(df_sample['t_latDeg'] - df_sample_filter['latDeg'], alpha = 1/3, bins = bins)

axes[3].plot(df_sample['lngDeg'])
axes[3].plot(df_sample_filter['lngDeg'])

axes[4].plot(df_sample['t_lngDeg'] - df_sample['lngDeg'])
axes[4].plot(df_sample['t_lngDeg'] - df_sample_filter['lngDeg'])

axes[5].hist(df_sample['t_lngDeg'] - df_sample['lngDeg'], alpha = 1/3, bins = bins)
axes[5].hist(df_sample['t_lngDeg'] - df_sample_filter['lngDeg'], alpha = 1/3, bins = bins)

axes[6].plot(df_sample['yawDeg'])
axes[6].plot(df_sample_filter['yawDeg'])

axes[7].plot(df_sample['UncalGyroZRadPerSec'])
axes[7].plot(df_sample_filter['UncalGyroZRadPerSec'])

In [ ]:
q_gain_list = np.arange(0.1, 2, 0.1)
r_gain_list = np.arange(0.1, 2, 0.1)

q_ = np.array([1,1,1,1,1])
r_ = np.array([1,1,1,1])
phones = df_train['phone'].unique()


result_list = []
iter_count = 0
for q_gain in q_gain_list:
    for r_gain in r_gain_list:
        iter_count += 1
        q = q_gain * q_
        r = r_gain * r_
#         sample_phone = np.random.choice(phones, 13, replace = False)
#         sample_index = df_train['phone'].apply(lambda x: x in sample_phone)
#         _, mean_before, score_before, mean_after, score_after = evaluate(df_train[sample_index], q, r)
        _, mean_before, score_before, mean_after, score_after = evaluate(df_train, q, r)
        result = [q, r, mean_after, mean_before, score_after, score_before]
        print(f"{q_gain}, {r_gain} : mean chagne - {mean_after - mean_before:.6f}, score change - {score_after - score_before:.6f}")
        print(f"after[{mean_after}, {score_after}], before[{mean_before}, {score_before}]")
        result_list.append(result)

  0%|          | 0/73 [00:00<?, ?it/s]

0.1, 0.1 : mean chagne - 0.039099, score change - -0.000208
after[3.8859468923498737, 5.287762571043874], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

0.1, 0.2 : mean chagne - 0.011640, score change - 0.012837
after[3.858488201422455, 5.300807302575015], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

...

  0%|          | 0/73 [00:00<?, ?it/s]

0.7000000000000001, 0.4 : mean chagne - -0.008953, score change - -0.004647
after[3.837895843444096, 5.283323991217206], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

0.7000000000000001, 0.5 : mean chagne - -0.018148, score change - -0.031671
after[3.828699986257111, 5.256299510983155], before[3.84684837499064, 5.2879706490841585]


  0%|          | 0/73 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(result_list, columns = ['q', 'r', 'mean_after', 'mean_before', 'score_after', 'score_before'])


In [ ]:
df['qgain'] = df['q'].apply(lambda x: x.mean())
df['rgain'] = df['r'].apply(lambda x: x.mean())
df['change_score'] = df['score_after'] - df['score_before']

In [ ]:
pt = df.pivot_table('change_score', 'qgain', 'rgain')
sns.heatmap(pt)

In [ ]:
qgain = df.loc[df['change_score'].min() == df['change_score'], 'qgain'].values
rgain = df.loc[df['change_score'].min() == df['change_score'], 'rgain'].values
print(qgain, rgain)

In [ ]:
_, mean_before, score_before, mean_after, score_after =  evaluate(df_train, qgain*q_, rgain*r_)
print(mean_before, score_before, mean_after, score_after)
print(f"mean change : {mean_after - mean_before:.4f}")
print(f"score change: {score_after - score_before:.4f}")

In [ ]:
submission = pd.read_csv("../input/google-smartphone-decimeter-challenge/sample_submission.csv")
submission = submission[['phone', 'millisSinceGpsEpoch']]

In [ ]:
result, _, _, _, _ =  evaluate(df_test, qgain*q_, rgain*r_, get_score=False)
result = result[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
submission = submission.merge(result, on = ['phone', 'millisSinceGpsEpoch'])

In [ ]:
submission.to_csv(f"./models/{'ComplexKalmanFilter1'}/result-{4}result.csv", index = False)